In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

path = "https://raw.githubusercontent.com/neohoft/Learn_pandas/master/Notebooks/Data/new_york_hotels.csv"

In [3]:
data = pd.read_csv(path, encoding="cp1251")
print("data.shape = {} rows, {} cols".format(*data.shape))

data.head(n=2)

data.shape = 20640 rows, 11 cols


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,id
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,1


In [3]:
def calculate_haversine(lat_x, lon_x, lat_y, lon_y):
    """
    Вычисление гаверсинусов.

    Parameters
    ----------
    lat_x: float
        Значение широты для начальной точки.

    lon_x: float
        Значение долготы для начальной точки.

    lat_y: float
        Значение широты для конечной точки.

    lon_y: float
        Значение долготы для конечной точки.

    Returns
    -------
    total_miles: float
        Значение гаверсинуса.

    """
    MILES = 3959
    lat_x, lon_x, lat_y, lon_y = map(
        np.deg2rad, [lat_x, lon_x, lat_y, lon_y]
    )
    dlat = lat_y - lat_x 
    dlon = lon_y - lon_x 
    a = np.sin(dlat/2)**2 + np.cos(lat_x) * np.cos(lat_y) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c

    return total_miles

## Базовое итерирование

In [4]:
def haversine_looping(df: pd.DataFrame) -> list:
    """
    Вычисление гаверсинусов в питоновском цикле.
    Очень медленная (по скорости работы) реализация, но
    достаточно легкая для написания кода.

    Parameters
    ----------
    df: pandas.core.frame.DataFrame
        Датафрейм с набором исходных данных.

    Returns
    -------
    distance_list: List[float]
        Список со значениями гаверсинусов.

    """
    distance_list = []
    for i in tqdm(range(0, len(df))):
        d = calculate_haversine(
            40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude']
        )
        distance_list.append(d)
    return distance_list

In [5]:
%%timeit

data["distance"] = haversine_looping(data)

100%|██████████| 1631/1631 [00:00<00:00, 1994.70it/s]

671 ms ± 62.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Итерирование с помощью встроенного метода .iterrows()

In [6]:
def haversine_iterrows(df: pd.DataFrame) -> list:
    """
    Вычисление гаверсинусов в питоновском цикле.
    Очень медленная (по скорости работы) реализация, но
    достаточно легкая для написания кода.

    Parameters
    ----------
    df: pandas.core.frame.DataFrame
        Датафрейм с набором исходных данных.

    Returns
    -------
    distance_list: List[float]
        Список со значениями гаверсинусов.

    """
    distance_list = []
    for index, row in tqdm(df.iterrows()):
        d = calculate_haversine(
            40.671, -73.985, row['latitude'], row['longitude']
        )
        distance_list.append(d)
    return distance_list

In [7]:
%%timeit

data["distance"] = haversine_iterrows(data)

1631it [00:00, 6426.98it/s]
1631it [00:00, 6351.02it/s]
1631it [00:00, 6732.28it/s]
1631it [00:00, 6709.50it/s]
1631it [00:00, 6942.15it/s]
1631it [00:00, 6745.14it/s]
1631it [00:00, 6782.63it/s]
1631it [00:00, 6800.11it/s]

246 ms ± 5.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Итерирование с помощью встроенного метода apply

In [8]:
%%timeit

data["distance"] = data.apply(
    lambda row: calculate_haversine(
        40.671, -73.985, row["latitude"], row["longitude"]), axis=1
)

78.9 ms ± 825 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Векторизация c помощью pandas.Series

In [9]:
%%timeit 

data["distance"] = calculate_haversine(
    40.671, -73.985, data["latitude"], data["longitude"]
)

2.41 ms ± 40.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Векторизация с помощью массивов NumPy

In [10]:
%%timeit

data["distance"] = calculate_haversine(
    40.671, -73.985, data["latitude"].values, data["longitude"].values
)

239 µs ± 4.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Эксперименты на более крупном наборе данных

In [11]:
n_samples = 100000

synthetic_data = pd.DataFrame({
    "latitude": np.random.randint(1, 179, size=n_samples),
    "longitude": np.random.randint(1, 179, size=n_samples),
})

synthetic_data["latitude"] = synthetic_data["latitude"] + np.random.rand(n_samples)
synthetic_data["longitude"] = synthetic_data["longitude"] + np.random.rand(n_samples)

## Итерирование с помощью цикла for

In [12]:
%%timeit

synthetic_data["distance"] = haversine_looping(synthetic_data)

100%|██████████| 100000/100000 [00:37<00:00, 2635.21it/s]

32.2 s ± 2.38 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Итерирование с помощью встроенного метода iterrows

In [13]:
%%timeit

synthetic_data["distance"] = haversine_iterrows(synthetic_data)

100000it [00:10, 9139.73it/s]
100000it [00:12, 8035.31it/s]
100000it [00:12, 7761.10it/s]
100000it [00:12, 7711.03it/s]
100000it [00:12, 8011.02it/s]
100000it [00:12, 7868.22it/s]
100000it [00:12, 7883.67it/s]
100000it [00:12, 7735.97it/s]

12.8 s ± 197 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Итерирование с помощью встроенного метода apply

In [14]:
%%timeit

synthetic_data["distance"] = synthetic_data.apply(
    lambda row: calculate_haversine(
        40.671, -73.985, row["latitude"], row["longitude"]), axis=1
)

6.03 s ± 270 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Векторизация с помощью pandas.Series

In [16]:
%%timeit 

synthetic_data["distance"] = calculate_haversine(
    40.671, -73.985, synthetic_data["latitude"], synthetic_data["longitude"]
)

16 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Векторизация с помощью массивов NumPy

In [17]:
%%timeit

synthetic_data["distance"] = calculate_haversine(
    40.671, -73.985, synthetic_data["latitude"].values, synthetic_data["longitude"].values
)

7.75 ms ± 95 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
